In [1]:
import pandas as pd
import yfinance as yf


In [2]:
#ASX Tickers
RAW_TICKER_ASX = pd.read_csv('https://www.asx.com.au/asx/research/ASXListedCompanies.csv',skiprows=1)
RAW_TICKER_ASX["Ticker"] = RAW_TICKER_ASX['ASX code']+".AX"
TICKER_ASX = list(RAW_TICKER_ASX['Ticker'].dropna())

In [3]:
# Nasdaq Tickers
#ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt
#ftp.nasdaqtrader.com/SymbolDirectory/otherlisted.txt

RAW_TICKER_NDQ = pd.read_csv('/Users/JZ/Downloads/nasdaqlisted.txt',sep="|")
RAW_TICKER_NDQ["Ticker"] = RAW_TICKER_NDQ['Symbol']
TICKER_NDQ = list(RAW_TICKER_NDQ['Ticker'].dropna())

In [4]:
#London Stock Exchange Tickers
RAW_TICKER_LSE = pd.read_excel (r'https://docs.londonstockexchange.com/sites/default/files/documents/list_of_sets_securities_14.xls', sheet_name='SETS',skiprows=3)
RAW_TICKER_LSE["Ticker"] = RAW_TICKER_LSE['Mnemonic']+".L"
TICKER_LSE = list(RAW_TICKER_LSE['Ticker'].dropna())

In [ ]:
#NYSE Tickers
#RAW_TICKER_NYSE = pd.read_excel (r'https://www.theice.com/publicdocs/data/NYSE_Equity_Index_Ticker_List.xlsx', sheet_name='SETS',skiprows=3)


In [5]:
#Append list for data extraction layer
TICKER_ALL = (TICKER_ASX + TICKER_NDQ + TICKER_LSE)
#Remove Duplicates
TICKER_ALL = list(dict.fromkeys(TICKER_ALL))

In [7]:
#Sample top 5 for testing code purposes
# TICKER_ASXs = TICKER_ASX[:5]
# TICKER_NDQs = TICKER_NDQ[:5]
# TICKER_LSEs = TICKER_LSE[:5]

# TICKER_ALL = (TICKER_ASXs + TICKER_NDQs + TICKER_LSEs)



In [12]:
#Download part 1: Company Information

INF_DF = pd.DataFrame()

def Info_Extract(TickName):    
    global INF_DF
    try:
        tick = yf.Ticker(TickName) 
        INF = tick.info
        INF_DF = INF_DF.append(INF, ignore_index=True) 
         
        print("finished Info:",TickName)
        
    except Exception:
        print("Error with Info:",TickName)
        pass
    
    return()

#for company in ASX_TICKER_LIST:
#    Info_Extract(company)



In [13]:
#Download part 2: financial data

FINANCIALS_DF = pd.DataFrame()

def Financials_Extract(TickName):
    global FINANCIALS_DF
    try:
        tick = yf.Ticker(TickName) 
        
        #income statement
        FIN_OG = tick.financials
        FIN_OG2 = FIN_OG.reset_index()
        FIN_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        FIN_OG3=FIN_OG2.melt(id_vars=["Metric"],var_name="Date",value_name="Value")
        FIN_OG3['TickName']=TickName
        FIN_OG3['Financial Data Type']="Income Statement"
        
        FINANCIALS_DF = FINANCIALS_DF.append(FIN_OG3, ignore_index=True)
         
        #balance sheet
        BS_OG = tick.balance_sheet
        BS_OG2 = BS_OG.reset_index()
        BS_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        BS_OG3=BS_OG2.melt(id_vars=["Metric"],var_name="Date",value_name="Value")
        BS_OG3['TickName']=TickName
        BS_OG3['Financial Data Type']="Balance Sheet"
        
        FINANCIALS_DF = FINANCIALS_DF.append(BS_OG3, ignore_index=True)
        
        #cashflow
        CF_OG = tick.cashflow
        CF_OG2 = CF_OG.reset_index()
        CF_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        CF_OG3=CF_OG2.melt(id_vars=["Metric"],var_name="Date",value_name="Value")
        CF_OG3['TickName']=TickName
        CF_OG3['Financial Data Type']="Cashflow"
        
        FINANCIALS_DF = FINANCIALS_DF.append(CF_OG3, ignore_index=True)
        
            
        print("finished Financial:",TickName)
        
    except Exception:
        print("Error with Financial:",TickName)
        pass

    return()

#for company in TICKER_ALL:
#    Financials_Extract(company)

In [19]:
#Download part 3: Download historical share prices

HIST_PRICE_DF = pd.DataFrame()

def Prices_Extract(TickName):
    
    global HIST_PRICE_DF

    try:
        tick = yf.Ticker(TickName) 
        
        hist_p = tick.history(period="6mo")
        hist_p2 = hist_p.reset_index()
        hist_p2.rename(columns = {'index':'Metric'}, inplace = True)
        
        hist_p2['TickName']=TickName

        HIST_PRICE_DF = HIST_PRICE_DF.append(hist_p2, ignore_index=True)
        
            
        print("finished Price:",TickName)
        
    except Exception:
        print("Error with Price:",TickName)
        pass

    return()


#for company in TICKER_ALL:
#    Prices_Extract(company)

In [15]:
#Part 4: historical dividend information

#DIV = tick.dividends  


HIST_DIV_DF = pd.DataFrame()

def Dividend_Extract(TickName):
    global HIST_DIV_DF
    try:
        tick = yf.Ticker(TickName) 
        
        #income statement
        DIV_OG = tick.dividends
        DIV_OG2 = DIV_OG.reset_index()
        DIV_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        DIV_OG2['TickName']=TickName
        
        HIST_DIV_DF = HIST_DIV_DF.append(DIV_OG2, ignore_index=True)
        
            
        print("finished Dividend",TickName)
        
    except Exception:
        print("Error with Dividend",TickName)
        pass

    return()


In [18]:
# Export all 3 sections to excel

for company in TICKER_ALL:
    Info_Extract(company)
INF_DF.to_excel('/Users/JZ/Downloads/Testlist/TEST_INFO.xlsx')


for company in TICKER_ALL:
    Financials_Extract(company)
FINANCIALS_DF.to_excel('/Users/JZ/Downloads/Testlist/TEST_FIN.xlsx')


for company in TICKER_ALL:
    Prices_Extract(company)
HIST_PRICE_DF.to_excel('/Users/JZ/Downloads/Testlist/TEST_Prices.xlsx')  

for company in TICKER_ALL:
    Dividend_Extract(company)
HIST_DIV_DF.to_excel('/Users/JZ/Downloads/Testlist/TEST_Div.xlsx')  



finished Info: MOQ.AX
finished Info: ONT.AX
finished Info: 14D.AX
finished Info: 1ST.AX
finished Info: T3D.AX
finished Info: AACG
finished Info: AACQU
finished Info: AAL
finished Info: AAME
finished Info: AAOI
finished Info: DGE.L
Error with Info: BA..L
finished Info: BATS.L
finished Info: IMB.L
finished Info: HSBA.L
finished Financial: MOQ.AX
finished Financial: ONT.AX
finished Financial: 14D.AX
finished Financial: 1ST.AX
finished Financial: T3D.AX
finished Financial: AACG
finished Financial: AACQU
finished Financial: AAL
finished Financial: AAME
finished Financial: AAOI
finished Financial: DGE.L
Error with Financial: BA..L
finished Financial: BATS.L
finished Financial: IMB.L
finished Financial: HSBA.L


NameError: name 'Prices_Extract' is not defined

In [ ]:
#Insert data
#DF_SHORT.to_sql('test', db_sql, if_exists='replace')

#pd.read_sql('select * from test', db_sql)